In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

In [3]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda, Flatten
from keras.layers import Convolution1D, Dense, Dropout, MaxPooling1D, LSTM
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
from keras import callbacks
from keras.callbacks import CSVLogger
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
import numpy as np
np.random.seed(0)

In [4]:
x = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/clean_5.csv")
x=x.dropna()

In [5]:
import pandas as pd
def clean_dataset(df):
    columns = df.columns
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

In [6]:
scaler = StandardScaler()

In [7]:
import netaddr 
ips = x[' Destination IP'].unique()
l = len(ips)
print('strating loop, length is',l)
for i in range(l):
  x[' Destination IP'] = x[' Destination IP'].replace(ips[i], int(netaddr.IPAddress(ips[i])))
print('loop over')

strating loop, length is 446
loop over


In [8]:
x = clean_dataset(x)

In [9]:
from sklearn import preprocessing

t = x.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(t)
normalized_df = pd.DataFrame(x_scaled)

In [10]:
y=x['Label']
x=x.drop(['Label'], axis=1)

In [11]:
y.nunique()

13

In [12]:
y = pd.get_dummies(y,prefix=['Label'])
y.head()

,['Label']_0.0,['Label']_1.0,['Label']_2.0,['Label']_3.0,['Label']_4.0,['Label']_5.0,['Label']_6.0,['Label']_7.0,['Label']_8.0,['Label']_9.0,['Label']_10.0,['Label']_11.0,['Label']_12.0
0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,0,0,0


In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(normalized_df, y, test_size=0.25, random_state=0)
y_train = np.array(y_train)

In [38]:
y_train = np.array(y_train)
y_test = np.array(y_test)
X_train = np.array(X_train)
X_test = np.array(X_test)
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))
X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))

In [39]:
print('xtrain={}, ytrain={}, xtest={}, ytest={}'.format(X_train.shape,y_train.shape,X_test.shape,y_test.shape))

xtrain=(798555, 38, 1), ytrain=(798555, 13), xtest=(266185, 38, 1), ytest=(266185, 13)


In [40]:
from keras import optimizers
sgd = optimizers.SGD(lr=0.009, decay=1e-6, momentum=0.9, nesterov=True)

In [31]:
model = Sequential()
model.add(Convolution1D(64, 3, activation="sigmoid", input_shape=(38,1)))
model.add(Convolution1D(32, 3, activation="sigmoid"))
#model.add(MaxPooling1D(pool_size=2))
#model.add(Convolution1D(128, 3, activation="relu"))
#model.add(Convolution1D(128, 3, activation="relu"))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(rate=0.3))
model.add(Flatten())
#model.add(Dense(30, activation="softmax"))
#model.add(Dense(10, activation="softmax"))
model.add(Dense(13))


In [32]:
model.compile(loss="mean_absolute_error", optimizer="SGD", metrics=['accuracy'])

model.fit(X_train, y_train, epochs=1, batch_size=100, validation_data=(X_test, y_test))

79856/79856 [==============================] - 296s 4ms/step - loss: 0.0991 - accuracy: 0.0793 - val_loss: 0.0772 - val_accuracy: 0.0858


In [30]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])

model.fit(X_train, y_train, epochs=1, batch_size=10, validation_data=(X_test, y_test))

79856/79856 [==============================] - 331s 4ms/step - loss: 0.7776 - accuracy: 0.0910 - val_loss: 0.7763 - val_accuracy: 0.0924


In [53]:
model2 = Sequential()
model2.add(Convolution1D(64, 3, activation="relu", input_shape=(38,1)))
model2.add(Convolution1D(32, 3, activation="relu"))
#model2.add(MaxPooling1D(pool_size=2))
#model2.add(Convolution1D(128, 3, activation="relu"))
#model2.add(Convolution1D(128, 3, activation="relu"))
model2.add(MaxPooling1D(pool_size=2))
#model2.add(Dropout(rate=0.3))
model2.add(Flatten())
#model2.add(Dense(30, activation="softmax"))
#model2.add(Dense(10, activation="softmax"))
model2.add(Dense(13))

In [54]:
model.compile(loss="mean_absolute_error", optimizer="SGD", metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=10, validation_data=(X_test, y_test))


Epoch 1/10
79856/79856 [==============================] - 290s 4ms/step - loss: 0.0774 - accuracy: 0.0796 - val_loss: 0.0772 - val_accuracy: 0.0920
Epoch 2/10
79856/79856 [==============================] - 272s 3ms/step - loss: 0.0772 - accuracy: 0.0785 - val_loss: 0.0772 - val_accuracy: 0.0852
Epoch 3/10
79856/79856 [==============================] - 272s 3ms/step - loss: 0.0772 - accuracy: 0.0780 - val_loss: 0.0771 - val_accuracy: 0.0819
Epoch 4/10
79856/79856 [==============================] - 272s 3ms/step - loss: 0.0772 - accuracy: 0.0769 - val_loss: 0.0772 - val_accuracy: 0.0926
Epoch 5/10
79856/79856 [==============================] - 273s 3ms/step - loss: 0.0772 - accuracy: 0.0769 - val_loss: 0.0772 - val_accuracy: 0.0920
Epoch 6/10
79856/79856 [==============================] - 272s 3ms/step - loss: 0.0772 - accuracy: 0.0777 - val_loss: 0.0772 - val_accuracy: 0.0885
Epoch 7/10
79856/79856 [==============================] - 270s 3ms/step - loss: 0.0772 - accuracy: 0.0781 - val_